In [3]:
import pandas as pd
import os

# Define the directory containing all emotion feature files
feature_dir = "C:/Users/bibek/Downloads/eeg main data txt/featurerequired"  # Directory containing the files

# List of all emotion files
emotion_files = ["fear_features.xlsx", "anger_features.xlsx", "normal_features.xlsx", 
                 "sad_features.xlsx", "happy_features.xlsx", "surprise_features.xlsx"]

# Initialize an empty list to store dataframes
all_features = []

# Loop through each file and read the data
for file in emotion_files:
    file_path = os.path.join(feature_dir, file)
    emotion_df = pd.read_excel(file_path)
    all_features.append(emotion_df)

# Combine all dataframes into one
combined_features = pd.concat(all_features, ignore_index=True)

# Reorder columns for better readability if needed
columns_order = ["Subject", "Emotion"] + [col for col in combined_features.columns if col not in ["Subject", "Emotion"]]
combined_features = combined_features[columns_order]

# Save the combined dataframe to a new Excel file
output_file = "C:/Users/bibek/Downloads/eeg main data txt/featurerequired/combined_features.xlsx"  # Desired output path
combined_features.to_excel(output_file, index=False)

print(f"Combined features saved to {output_file}")


Combined features saved to C:/Users/bibek/Downloads/eeg main data txt/featurerequired/combined_features.xlsx


In [20]:
import os
import pandas as pd

# Define the folder path for the emotion
emotion_folder = r'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\surprise'
output_file = r'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\surprise_combined.csv'

# Initialize an empty list to store dataframes
dataframes = []

# Loop through all files in the folder
for file_name in os.listdir(emotion_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(emotion_folder, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Add a column to identify the subject (optional but helpful)
        df['Subject'] = file_name.replace('.csv', '')
        
        # Append the dataframe to the list
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined dataframe to a single CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined file saved at: {output_file}")


Combined file saved at: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\surprise_combined.csv


In [26]:
import os
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import skew, kurtosis
from scipy.integrate import simps
import pywt

# Function to calculate band power
def bandpower(psd, freqs, band):
    band_idx = np.logical_and(freqs >= band[0], freqs <= band[1])
    return simps(psd[:, band_idx], freqs[band_idx], axis=1)

# Feature extraction functions
def extract_features(data, fs=256):
    features = {}
    
    # Time-domain features
    features['mean'] = np.mean(data, axis=1)
    features['std'] = np.std(data, axis=1)
    features['skewness'] = skew(data, axis=1)
    features['kurtosis'] = kurtosis(data, axis=1)
    features['peak_to_peak'] = np.ptp(data, axis=1)

    # Frequency-domain features (using Welch's method)
    freqs, psd = welch(data, fs, nperseg=256, axis=1)
    
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 50)

    features['delta_power'] = bandpower(psd, freqs, delta_band)
    features['theta_power'] = bandpower(psd, freqs, theta_band)
    features['alpha_power'] = bandpower(psd, freqs, alpha_band)
    features['beta_power'] = bandpower(psd, freqs, beta_band)
    features['gamma_power'] = bandpower(psd, freqs, gamma_band)

    # Nonlinear features
    features['entropy'] = -np.sum(psd * np.log(psd + 1e-10), axis=1)  # Spectral entropy

    # Time-frequency features using wavelet transform
    coeffs = pywt.wavedec(data, 'db4', axis=1, level=5)
    for i, c in enumerate(coeffs):
        features[f'wavelet_level_{i+1}_mean'] = np.mean(c, axis=1)
        features[f'wavelet_level_{i+1}_std'] = np.std(c, axis=1)

    return features

# Process a dataset and extract features
def process_dataset(file_path, emotion_label):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Extract numerical EEG data and subject info
    eeg_data = data.iloc[:, :-1].values  # All columns except 'Subject'
    subjects = data['Subject']

    # Extract features
    features = extract_features(eeg_data)

    # Combine features into a DataFrame
    features_df = pd.DataFrame(features)
    features_df['Subject'] = subjects
    features_df['Emotion'] = emotion_label

    return features_df

# Directory and filenames
base_dir = r'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine'
file_emotions = {
    'surprise': 'surprise_preprocessed',
    'fear': 'fear_preprocessed',
    'normal': 'normal_preprocessed',
    'anger': 'anger_preprocessed',
    'sad': 'sad_preprocessed',
    'happy': 'happy_preprocessed'
}

# Process all files and combine features
all_features = []
for emotion, filename in file_emotions.items():
    file_path = os.path.join(base_dir, filename)
    emotion_features = process_dataset(file_path, emotion)
    all_features.append(emotion_features)

# Concatenate all features into a single DataFrame
all_features_df = pd.concat(all_features, ignore_index=True)

# Save the features to a CSV file
output_path = os.path.join(base_dir, 'extracted_eeg_features.csv')
all_features_df.to_csv(output_path, index=False)

print(f"Feature extraction completed. Saved to '{output_path}'.")
import os
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import skew, kurtosis
from scipy.integrate import simps
import pywt

# Function to calculate band power
def bandpower(psd, freqs, band):
    band_idx = np.logical_and(freqs >= band[0], freqs <= band[1])
    return simps(psd[:, band_idx], freqs[band_idx], axis=1)

# Feature extraction functions
def extract_features(data, fs=256):
    features = {}
    
    # Time-domain features
    features['mean'] = np.mean(data, axis=1)
    features['std'] = np.std(data, axis=1)
    features['skewness'] = skew(data, axis=1)
    features['kurtosis'] = kurtosis(data, axis=1)
    features['peak_to_peak'] = np.ptp(data, axis=1)

    # Frequency-domain features (using Welch's method)
    freqs, psd = welch(data, fs, nperseg=256, axis=1)
    
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 50)

    features['delta_power'] = bandpower(psd, freqs, delta_band)
    features['theta_power'] = bandpower(psd, freqs, theta_band)
    features['alpha_power'] = bandpower(psd, freqs, alpha_band)
    features['beta_power'] = bandpower(psd, freqs, beta_band)
    features['gamma_power'] = bandpower(psd, freqs, gamma_band)

    # Nonlinear features
    features['entropy'] = -np.sum(psd * np.log(psd + 1e-10), axis=1)  # Spectral entropy

    # Time-frequency features using wavelet transform
    coeffs = pywt.wavedec(data, 'db4', axis=1, level=5)
    for i, c in enumerate(coeffs):
        features[f'wavelet_level_{i+1}_mean'] = np.mean(c, axis=1)
        features[f'wavelet_level_{i+1}_std'] = np.std(c, axis=1)

    return features

# Process a dataset and extract features
def process_dataset(file_path, emotion_label):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Extract numerical EEG data and subject info
    eeg_data = data.iloc[:, :-1].values  # All columns except 'Subject'
    subjects = data['Subject']

    # Extract features
    features = extract_features(eeg_data)

    # Combine features into a DataFrame
    features_df = pd.DataFrame(features)
    features_df['Subject'] = subjects
    features_df['Emotion'] = emotion_label

    return features_df

# Directory and filenames
base_dir = r'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine'
file_emotions = {
    'surprise': 'surprise_preprocessed.csv',
    'fear': 'fear_preprocessed.csv',
    'normal': 'normal_preprocessed.csv',
    'anger': 'anger_preprocessed.csv',
    'sad': 'sad_preprocessed.csv',
    'happy': 'happy_preprocessed.csv'
}

# Process all files and combine features
all_features = []
for emotion, filename in file_emotions.items():
    file_path = os.path.join(base_dir, filename)
    emotion_features = process_dataset(file_path, emotion)
    all_features.append(emotion_features)

# Concatenate all features into a single DataFrame
all_features_df = pd.concat(all_features, ignore_index=True)

# Save the features to a CSV file
output_path = os.path.join(base_dir, 'extracted_eeg_features.csv')
all_features_df.to_csv(output_path, index=False)

print(f"Feature extraction completed. Saved to '{output_path}'.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bibek\\Downloads\\eeg main data txt\\preprocessednew\\preprocessedcombine\\surprise_preprocessed'

In [28]:
import os
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import skew, kurtosis
from scipy.integrate import simps
import pywt

# Function to calculate band power
def bandpower(psd, freqs, band):
    band_idx = np.logical_and(freqs >= band[0], freqs <= band[1])
    return simps(psd[:, band_idx], freqs[band_idx], axis=1)

# Feature extraction functions
def extract_features(data, fs=256):
    features = {}
    
    # Time-domain features
    features['mean'] = np.mean(data, axis=1)
    features['std'] = np.std(data, axis=1)
    features['skewness'] = skew(data, axis=1)
    features['kurtosis'] = kurtosis(data, axis=1)
    features['peak_to_peak'] = np.ptp(data, axis=1)

    # Frequency-domain features (using Welch's method)
    freqs, psd = welch(data, fs, nperseg=256, axis=1)
    
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 50)

    features['delta_power'] = bandpower(psd, freqs, delta_band)
    features['theta_power'] = bandpower(psd, freqs, theta_band)
    features['alpha_power'] = bandpower(psd, freqs, alpha_band)
    features['beta_power'] = bandpower(psd, freqs, beta_band)
    features['gamma_power'] = bandpower(psd, freqs, gamma_band)

    # Nonlinear features
    features['entropy'] = -np.sum(psd * np.log(psd + 1e-10), axis=1)  # Spectral entropy

    # Time-frequency features using wavelet transform
    coeffs = pywt.wavedec(data, 'db4', axis=1, level=5)
    for i, c in enumerate(coeffs):
        features[f'wavelet_level_{i+1}_mean'] = np.mean(c, axis=1)
        features[f'wavelet_level_{i+1}_std'] = np.std(c, axis=1)

    return features

# Process a dataset and extract features
def process_dataset(file_path, emotion_label):
    # Log the file being processed
    print(f"Processing file: {file_path}")

    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Extract numerical EEG data and subject info
    eeg_data = data.iloc[:, :-1].values  # All columns except 'Subject'
    subjects = data['Subject']

    # Extract features
    features = extract_features(eeg_data)

    # Combine features into a DataFrame
    features_df = pd.DataFrame(features)
    features_df['Subject'] = subjects
    features_df['Emotion'] = emotion_label

    return features_df

# Directory and filenames
base_dir = r'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine'
file_emotions = {
    'surprise': 'surprise_preprocessed.csv',
    'fear': 'fear_preprocessed.csv',
    'normal': 'normal_preprocessed.csv',
    'anger': 'anger_preprocessed.csv',
    'sad': 'sad_preprocessed.csv',
    'happy': 'happy_preprocessed.csv'
}

# Process all files and combine features
all_features = []
for emotion, filename in file_emotions.items():
    file_path = os.path.join(base_dir, filename)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    emotion_features = process_dataset(file_path, emotion)
    all_features.append(emotion_features)

# Concatenate all features into a single DataFrame
if all_features:
    all_features_df = pd.concat(all_features, ignore_index=True)

    # Save the features to a CSV file
    output_path = os.path.join(base_dir, 'extracted_eeg_features.csv')
    all_features_df.to_csv(output_path, index=False)

    print(f"Feature extraction completed. Saved to '{output_path}'.")
else:
    print("No files were processed. Please check the file paths.")

Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\surprise_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\3174670781.py:12: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return simps(psd[:, band_idx], freqs[band_idx], axis=1)


Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\fear_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\3174670781.py:12: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return simps(psd[:, band_idx], freqs[band_idx], axis=1)


Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\normal_preprocessed.csv


...

Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\sad_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\3174670781.py:12: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return simps(psd[:, band_idx], freqs[band_idx], axis=1)


Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\happy_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\3174670781.py:12: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  return simps(psd[:, band_idx], freqs[band_idx], axis=1)


Feature extraction completed. Saved to 'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\extracted_eeg_features.csv'.


In [30]:
import os
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import skew, kurtosis
from scipy.integrate import simpson
import pywt

# Function to calculate band power
def bandpower(psd, freqs, band):
    band_idx = np.logical_and(freqs >= band[0], freqs <= band[1])
    return simpson(psd[:, band_idx], freqs[band_idx], axis=1)

# Feature extraction functions
def extract_features(data, fs=256):
    features = {}
    
    # Time-domain features
    features['mean'] = np.mean(data, axis=1)
    features['std'] = np.std(data, axis=1)
    features['skewness'] = skew(data, axis=1)
    features['kurtosis'] = kurtosis(data, axis=1)
    features['peak_to_peak'] = np.ptp(data, axis=1)

    # Frequency-domain features (using Welch's method)
    freqs, psd = welch(data, fs, nperseg=256, axis=1)
    
    delta_band = (0.5, 4)
    theta_band = (4, 8)
    alpha_band = (8, 13)
    beta_band = (13, 30)
    gamma_band = (30, 50)

    features['delta_power'] = bandpower(psd, freqs, delta_band)
    features['theta_power'] = bandpower(psd, freqs, theta_band)
    features['alpha_power'] = bandpower(psd, freqs, alpha_band)
    features['beta_power'] = bandpower(psd, freqs, beta_band)
    features['gamma_power'] = bandpower(psd, freqs, gamma_band)

    # Nonlinear features
    features['entropy'] = -np.sum(psd * np.log(psd + 1e-10), axis=1)  # Spectral entropy

    # Time-frequency features using wavelet transform
    coeffs = pywt.wavedec(data, 'db4', axis=1, level=5)
    for i, c in enumerate(coeffs):
        features[f'wavelet_level_{i+1}_mean'] = np.mean(c, axis=1)
        features[f'wavelet_level_{i+1}_std'] = np.std(c, axis=1)

    return features

# Process a dataset and extract features
def process_dataset(file_path, emotion_label):
    # Log the file being processed
    print(f"Processing file: {file_path}")

    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Extract numerical EEG data and subject info
    eeg_data = data.iloc[:, :-1].values  # All columns except 'Subject'
    subjects = data['Subject']

    # Extract features
    features = extract_features(eeg_data)

    # Combine features into a DataFrame
    features_df = pd.DataFrame(features)
    features_df['Subject'] = subjects
    features_df['Emotion'] = emotion_label

    return features_df

# Directory and filenames
base_dir = r'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine'
file_emotions = {
    'surprise': 'surprise_preprocessed.csv',
    'fear': 'fear_preprocessed.csv',
    'normal': 'normal_preprocessed.csv',
    'anger': 'anger_preprocessed.csv',
    'sad': 'sad_preprocessed.csv',
    'happy': 'happy_preprocessed.csv'
}

# Process all files and combine features
all_features = []
for emotion, filename in file_emotions.items():
    file_path = os.path.join(base_dir, filename)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    emotion_features = process_dataset(file_path, emotion)
    all_features.append(emotion_features)

# Concatenate all features into a single DataFrame
if all_features:
    all_features_df = pd.concat(all_features, ignore_index=True)

    # Save the features to a CSV file
    output_path = os.path.join(base_dir, 'extracted_eeg_features.csv')
    all_features_df.to_csv(output_path, index=False)

    print(f"Feature extraction completed. Saved to '{output_path}'.")
else:
    print("No files were processed. Please check the file paths.")


Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\surprise_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[1. 2. 3. 4.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[4. 5. 6. 7. 8.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[ 8.  9. 10. 11. 12. 13.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, b

Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\fear_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[1. 2. 3. 4.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[4. 5. 6. 7. 8.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[ 8.  9. 10. 11. 12. 13.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, b

Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\normal_preprocessed.csv


...

Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\sad_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[1. 2. 3. 4.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[4. 5. 6. 7. 8.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[ 8.  9. 10. 11. 12. 13.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, b

Processing file: C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\happy_preprocessed.csv


C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[1. 2. 3. 4.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[4. 5. 6. 7. 8.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, band_idx], freqs[band_idx], axis=1)
C:\Users\bibek\AppData\Local\Temp\ipykernel_4052\249776699.py:12: DeprecationWarning: You are passing x=[ 8.  9. 10. 11. 12. 13.] as a positional argument. Please change your invocation to use keyword arguments. From SciPy 1.14, passing these as positional arguments will result in an error.
  return simpson(psd[:, b

Feature extraction completed. Saved to 'C:\Users\bibek\Downloads\eeg main data txt\preprocessednew\preprocessedcombine\extracted_eeg_features.csv'.


In [32]:
import pandas as pd

# Load the dataset
file_path = 'C:/Users/bibek/Downloads/PROJECT/EEG/features/featurescombined/extracted_eeg_features.csv'
eeg_features_df = pd.read_csv(file_path)

# Group by 'Emotion' and calculate the mean for each feature
emotion_feature_means = eeg_features_df.groupby('Emotion').mean()

# Reset the index to make 'Emotion' a column
emotion_feature_means.reset_index(inplace=True)

# Save the result to a new CSV file
output_file_path = 'C:/Users/bibek/Downloads/PROJECT/EEG/features/featurescombined/emotion_feature_means.csv'
emotion_feature_means.to_csv(output_file_path, index=False)

print(f"Combined feature means for each emotion saved to: {output_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/bibek/Downloads/PROJECT/EEG/features/featurescombined/extracted_eeg_features.csv'

In [36]:
import pandas as pd

# Load the dataset
file_path = 'C:/Users/bibek/Downloads/PROJECT/features'
eeg_features_df = pd.read_csv(file_path)

# Group by 'Emotion' and calculate the mean for each feature
emotion_feature_means = eeg_features_df.groupby('Emotion').mean()

# Reset the index to make 'Emotion' a column
emotion_feature_means.reset_index(inplace=True)

# Save the result to a new CSV file
output_file_path = 'C:/Users/bibek/Downloads/PROJECT/emotion_feature_means.csv'
emotion_feature_means.to_csv(output_file_path, index=False)

print(f"Combined feature means for each emotion saved to: {output_file_path}")
import pandas as pd

# Load the dataset
file_path = 'C:/Users/bibek/Downloads/PROJECT/features.csv'
eeg_features_df = pd.read_csv(file_path)

# Group by 'Emotion' and calculate the mean for each feature
emotion_feature_means = eeg_features_df.groupby('Emotion').mean()

# Reset the index to make 'Emotion' a column
emotion_feature_means.reset_index(inplace=True)

# Save the result to a new CSV file
output_file_path = 'C:/Users/bibek/Downloads/PROJECT/emotion_feature_means.csv'
emotion_feature_means.to_csv(output_file_path, index=False)

print(f"Combined feature means for each emotion saved to: {output_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/bibek/Downloads/PROJECT/features'

In [38]:
import pandas as pd

# Load the dataset
file_path = 'C:/Users/bibek/Downloads/PROJECT/features.csv'
eeg_features_df = pd.read_csv(file_path)

# Group by 'Emotion' and calculate the mean for each feature
emotion_feature_means = eeg_features_df.groupby('Emotion').mean()

# Reset the index to make 'Emotion' a column
emotion_feature_means.reset_index(inplace=True)

# Save the result to a new CSV file
output_file_path = 'C:/Users/bibek/Downloads/PROJECT/emotion_feature_means.csv'
emotion_feature_means.to_csv(output_file_path, index=False)

print(f"Combined feature means for each emotion saved to: {output_file_path}")


TypeError: agg function failed [how->mean,dtype->object]

In [44]:
import pandas as pd

# Load the dataset
file_path = 'C:\Users\bibek\Downloads\PROJECT\feature.csv'
eeg_features_df = pd.read_csv(file_path)

# Group by 'Emotion' and calculate the mean for each feature
emotion_feature_means = eeg_features_df.groupby('Emotion').mean()

# Reset the index to make 'Emotion' a column
emotion_feature_means.reset_index(inplace=True)

# Save the result to a new CSV file
output_file_path = 'C:/Users/bibek/Downloads/PROJECT/emotion_feature_means.csv'
emotion_feature_means.to_csv(output_file_path, index=False)

print(f"Combined feature means for each emotion saved to: {output_file_path}")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (4173987518.py, line 4)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3023995565.py, line 4)

In [70]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\bibek\Downloads\PROJECT\features.csv"
eeg_features_df = pd.read_csv(file_path)

# Group by 'Emotion' and calculate the mean for each feature
emotion_feature_means = eeg_features_df.groupby('Emotion').mean()

# Reset the index to make 'Emotion' a column
emotion_feature_means.reset_index(inplace=True)

# Save the result to a new CSV file
output_file_path = r"C:\Users\bibek\Downloads\PROJECT\emotion_feature_means.csv"
emotion_feature_means.to_csv(output_file_path, index=False)

print(f"Combined feature means for each emotion saved to: {output_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bibek\\Downloads\\PROJECT\\features.csv'

In [81]:
import os
import pandas as pd

# Define the file path
file_path = r"C:\Users\bibek\Downloads\PROJECT\feature\features.csv"

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_path = r"C:\Users\bibek\Downloads\PROJECT\feature\emotion_feature_means.csv"
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")


File not found: C:\Users\bibek\Downloads\PROJECT\feature\features.csv


In [87]:
import os
import pandas as pd

# Define the file path
file_path = r"C:\Users\bibek\Downloads\PROJECT\feature"

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_path = r"C:\Users\bibek\Downloads\PROJECT\feature\emotion_feature_means.csv"
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")
    

An error occurred while processing the file: [Errno 13] Permission denied: 'C:\\Users\\bibek\\Downloads\\PROJECT\\feature'


In [89]:
import os
import pandas as pd

# Define the file path
file_path = r"C:\Users\bibek\Downloads\PROJECT\feature"

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_path = r"C:\Users\bibek\Downloads\PROJECT\feature\emotion_feature_means.csv"
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")
    

An error occurred while processing the file: [Errno 13] Permission denied: 'C:\\Users\\bibek\\Downloads\\PROJECT\\feature'


In [91]:
import os
import pandas as pd

# Define the folder location
folder_path = r"C:\Users\bibek\Downloads\PROJECT\feature"
file_name = "features.csv"  # Specify the file name
file_path = os.path.join(folder_path, file_name)  # Combine folder path and file name

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_name = "emotion_feature_means.csv"  # Output file name
        output_file_path = os.path.join(folder_path, output_file_name)  # Save in the same folder
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")


File not found: C:\Users\bibek\Downloads\PROJECT\feature\feature.csv


In [93]:
import os
import pandas as pd

# Define the folder location
folder_path = r"C:\Users\bibek\Downloads\PROJECT\feature"
file_name = "features.csv"  # Specify the file name
file_path = os.path.join(folder_path, file_name)  # Combine folder path and file name

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_name = "emotion_feature_means.csv"  # Output file name
        output_file_path = os.path.join(folder_path, output_file_name)  # Save in the same folder
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")


File not found: C:\Users\bibek\Downloads\PROJECT\feature\features.csv


In [97]:
import os
import pandas as pd

# Define the folder location
folder_path = r"C:\Users\bibek\Downloads\PROJECT\feature"
file_name = "feature.csv"  # Adjust the name based on your file
file_path = os.path.join(folder_path, file_name)

# Debug: List files in the directory
print("Files in directory:", os.listdir(folder_path))

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_name = "emotion_feature_means.csv"
        output_file_path = os.path.join(folder_path, output_file_name)
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")


Files in directory: ['features.csv']
File not found: C:\Users\bibek\Downloads\PROJECT\feature\feature.csv


In [103]:
import os
import pandas as pd

# Define the folder location
folder_path = r"C:\Users\bibek\Downloads\PROJECT\feature"
file_name = "feature.csv"  # Adjust the name based on your file
file_path = os.path.join(folder_path, file_name)

# Debug: List files in the directory
print("Files in directory:", os.listdir(folder_path))

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Group by 'Emotion' and calculate the mean for each feature
        emotion_feature_means = eeg_features_df.groupby('Emotion').mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_name = "emotion_feature_means.csv"
        output_file_path = os.path.join(folder_path, output_file_name)
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")
    

Files in directory: ['feature.csv.csv']
File not found: C:\Users\bibek\Downloads\PROJECT\feature\feature.csv


In [105]:
import os
import pandas as pd

# Define the folder location
folder_path = r"C:\Users\bibek\Downloads\PROJECT\feature"
file_name = "feature.csv"
file_path = os.path.join(folder_path, file_name)

# Check if the file exists
if os.path.exists(file_path):
    try:
        # Load the dataset
        eeg_features_df = pd.read_csv(file_path)
        
        # Debug: Display data types
        print("Column data types:\n", eeg_features_df.dtypes)
        
        # Select numeric columns only
        numeric_columns = eeg_features_df.select_dtypes(include='number').columns
        print("Numeric columns:\n", numeric_columns)
        
        # Group by 'Emotion' and calculate the mean for each numeric feature
        emotion_feature_means = eeg_features_df.groupby('Emotion')[numeric_columns].mean()
        
        # Reset the index to make 'Emotion' a column
        emotion_feature_means.reset_index(inplace=True)
        
        # Define the output file path
        output_file_name = "emotion_feature_means.csv"
        output_file_path = os.path.join(folder_path, output_file_name)
        
        # Save the result to a new CSV file
        emotion_feature_means.to_csv(output_file_path, index=False)
        
        print(f"Combined feature means for each emotion saved to: {output_file_path}")
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
else:
    print(f"File not found: {file_path}")


File not found: C:\Users\bibek\Downloads\PROJECT\feature\feature.csv
